In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import math
import numpy as np
import pandas as pd
from tqdm import tqdm
from pathlib import Path
import sys
import os
import datetime

In [3]:
# Usando Google Colab:
'''
from google.colab import drive
drive.mount('/content/drive')

pasta = str("/content/drive/MyDrive/Python/TCC-Cloud/Data/")
print(pasta)

preços = pd.read_csv(pasta + "Preços.csv",index_col='Data',parse_dates=True)
semestres = pd.read_csv(pasta + "Semestres.csv",parse_dates=True)

Datas = pd.DataFrame(preços.index)
Datas['indice'] = Datas.index
log_data = np.log(preços)
'''

'\nfrom google.colab import drive\ndrive.mount(\'/content/drive\')\n\npasta = str("/content/drive/MyDrive/Python/TCC-Cloud/Data/")\nprint(pasta)\n\npreços = pd.read_csv(pasta + "Preços.csv",index_col=\'Data\',parse_dates=True)\nsemestres = pd.read_csv(pasta + "Semestres.csv",parse_dates=True)\n\nDatas = pd.DataFrame(preços.index)\nDatas[\'indice\'] = Datas.index\nlog_data = np.log(preços)\n'

In [4]:
# Especificando o caminho padrão do projeto no computador:
pasta = str(Path.cwd().parents[0])
print(pasta)

preços = pd.read_csv(pasta + "/Data/Preços.csv",index_col='Data',parse_dates=True)
semestres = pd.read_csv(pasta + "/Data/Semestres.csv",parse_dates=True)

Datas = pd.DataFrame(preços.index)
Datas['indice'] = Datas.index
log_data = np.log(preços)

/mnt/c/Users/MarceloPolecram/Python/MEU_TCC


In [5]:
def calculate_profit(pair, spread, threshold, par1, par2, resumo, semester):

    date_format = "%Y-%m-%d"

    log_ret = spread.diff() # log return eh o incremento
    dias = spread.index

    portfolio_return = []
    pos = 0 # 0: sem posição aberta
            # 1: Comprei o meu portfolio h = (1,-gamma)
            # -1: Vendi o meu portfolio h = -(1,-gamma)

    dias_abertura = []
    dias_fechamento = []

    count = 0
    dia_abertura = 0
    dia_fechamento = 0

    closing_threshold = 0.0

    for i in range(1, len(spread)):
        if (spread.iloc[i][0] > threshold) and (pos == 0):
            # Posição fechada no par e com sinal short 1st e long 2nd
            pos = -1

            count += 1
            dia_abertura = dias[i] - dias[0]
            retornos_op = []


        elif (spread.iloc[i][0] < -threshold)  and (pos == 0):
            # Posição fechada no par e com sinal de long 1st e short 2nd
            pos = 1

            count += 1
            dia_abertura = dias[i] - dias[0]
            retornos_op = []

        else:
            if (pos != 0) and ((dias[i] - dias[0] - dia_abertura) == duration_limit):
                #Fechando operações maiores que 50 dias
                portfolio_return.append(log_ret.iloc[i][0]*pos)
                pos = 0
                dia_fechamento = dias[i] - dias[0]
                delta_dias = dia_fechamento - dia_abertura
                retornos_op.append(log_ret.iloc[i][0]*pos)
                retorno_op = pd.Series(retornos_op).sum()


                resumo.append([count, semester, dia_abertura, dia_fechamento, delta_dias, retorno_op, par1, par2, True])

            elif (pos == 1) and (spread.iloc[i][0] >= -closing_threshold):
                # Posição vendida aberta no par e com sinal de convergência
                portfolio_return.append(log_ret.iloc[i][0]*pos)
                pos = 0

                dia_fechamento = dias[i] - dias[0]
                delta_dias = dia_fechamento - dia_abertura
                retornos_op.append(log_ret.iloc[i][0]*pos)
                retorno_op = pd.Series(retornos_op).sum()


                resumo.append([count, semester, dia_abertura, dia_fechamento, delta_dias, retorno_op, par1, par2, True])

            elif (pos == -1) and (spread.iloc[i][0] <= closing_threshold):
                portfolio_return.append(log_ret.iloc[i][0]*pos)
                pos = 0

                dia_fechamento = dias[i] - dias[0]
                delta_dias = dia_fechamento - dia_abertura
                retornos_op.append(log_ret.iloc[i][0]*pos)
                retorno_op = pd.Series(retornos_op).sum()


                resumo.append([count, semester, dia_abertura, dia_fechamento, delta_dias, retorno_op, par1, par2, True])

            elif (pos == 1) and (spread.iloc[i][0] < -closing_threshold):
                # Posição vendidada aberta no par aberta e sem convergência
                portfolio_return.append(log_ret.iloc[i][0]*pos)
                retornos_op.append(log_ret.iloc[i][0]*pos)


            elif (pos == -1) and (spread.iloc[i][0] > closing_threshold):
                portfolio_return.append(log_ret.iloc[i][0]*pos)
                retornos_op.append(log_ret.iloc[i][0]*pos)


            else:
                # Sem posição aberta e nem sinal de entrada

                if pos != 0:
                    dia_fechamento = dias[i] - dias[0]
                    delta_dias = dia_fechamento - dia_abertura
                    retornos_op.append(log_ret.iloc[i][0]*pos)
                    retorno_op = pd.Series(retornos_op).sum()

                    resumo.append([count, semester, dia_abertura, dia_fechamento, delta_dias, retorno_op, par1, par2, True])

                pos = 0

    if pos != 0:
        # Operação sem convergência
        pos = 0

        dia_fechamento = dias[i] - dias[0]
        delta_dias = dia_fechamento - dia_abertura
        retorno_op = pd.Series(retornos_op).sum()
        print(f"Par {par1}-{par2} sem convergência, retorno_op: {retorno_op}")
        resumo.append([count, semester, dia_abertura, dia_fechamento, delta_dias, retorno_op, par1, par2, False])

    #print(f"Total return: {sum(pair_returns)} | Pair returns: {pair_returns}")
    #print(f"Conversão do par: {pos}")
    total_ret = pd.Series(portfolio_return).sum()

    return total_ret, resumo

In [6]:
daylag = 0
wi_update = 1
years = 2023 - 1995

no_pairs = 5
days, num_assets = np.shape(preços)
trading_costs = 0
# buy/sell (percentage cost for opening and closing pairs: 0.001, 0.002, for example)
percentage_costs = 0
# set whether (0) or not (2) positive trading volume is required for opening/closing a pair
trade_req = 0
# Choose how much loss we are willing to accePt on a given pair, compared to 1, i.e, 0.93 = 7% stop loss
stop_loss = float('-inf')

# Choose how much gain we are willing to accePt on a given pair, compared to 1, i.e 1.10 = 10% stop gain
stop_gain = float('inf')
opening_threshold = 2
closing_threshold = 0.0

duration_limit = 50

In [7]:
big_loop = 0
i = 0
pairs = []
past_days= 0

for big_loop in range(0, len(semestres) - 3):
    print(f"Starting period {big_loop} | Past days: {past_days}")

    # Listando os dias dos intervalos
    inicio = (Datas == semestres['Data'][big_loop]).query("Data == True").index[0]
    twelve_months = (Datas == semestres['Data'][big_loop + 2]).query("Data == True").index[0]
    six_months = (Datas == semestres['Data'][big_loop + 3]).query("Data == True").index[0]

    # Limpeza das ações não listadas no período
    ps = log_data.iloc[inicio:twelve_months,:].dropna(how="any",axis=1) # Só usar se der erro
    listed = ps.iloc[0]
    listed_num = np.sum(listed.value_counts())
    print(f"Listed stocks for the period: {listed_num}")

    log_data_filtrado = log_data.columns.isin(listed.index)
    listed_indexes = np.where(log_data_filtrado > 0)[0]
    listed_stocks = listed.index

    # Preços no intervalo

    # ----------------------------------------------------
    # Calc SSEs
    # ----------------------------------------------------

    sse = np.zeros((listed_num, listed_num))
    for j in tqdm(range(0, listed_num- 1)):
        for k in range(j+1, listed_num):
            sse[j, k] = sum(np.power(ps.iloc[:, j]-ps.iloc[:, k], 2))

    print(f"SSE shape: {np.shape(sse)}")
    pd.DataFrame(sse).to_csv("SSE.csv", header=[
        str(i) for i in listed_indexes], index=False)

    # ----------------------------------------------------
    # Find min SSEs
    # ----------------------------------------------------

    max_SSE = np.nanmax(sse) + 1
    min_SSE = np.zeros((no_pairs, 1))
    min_SSE_ro = np.zeros((1, no_pairs))
    min_SSE_co = np.zeros((1, no_pairs))

    # print(f"Initial Max SSE: {max_SSE}")

    for ii in range(0, no_pairs):
        t_SSE = max_SSE
        for k in range(0, listed_num-1):
            for l in range(k+1, listed_num):
                if sse[k, l] > 0 and sse[k, l] < t_SSE:
                    # print(f"New minimum found at ({k},{l})")
                    t_SSE = sse[k, l]  # new minimum found

        print(f"Minimum SSE = {t_SSE}")

        if t_SSE == max_SSE:
            print("Error")

        ro, co = np.where(sse == t_SSE)
        ro = ro[0]
        co = co[0]

        min_SSE[ii, 0] = sse[ro, co]

        pair_dict = {'Stock A': log_data.columns[listed_indexes[ro]],'Stock B': log_data.columns[listed_indexes[co]],'SSE' : min_SSE[ii, 0],'Semestre' : big_loop}
        pairs.append(pair_dict)
        pd.DataFrame(pairs)

        sse[ro, co] = max_SSE  # prevent re-selection

    past_days =  twelve_months

pairs= pd.DataFrame(pairs)

Starting period 0 | Past days: 0
Listed stocks for the period: 57


100%|██████████████████████████████████████████████████████████████████████████████████| 56/56 [00:00<00:00, 141.59it/s]


SSE shape: (57, 57)
Minimum SSE = 1.1470111572247645
Minimum SSE = 1.2292074671340252
Minimum SSE = 1.4172821904116257
Minimum SSE = 1.469926115829889
Minimum SSE = 1.6763074444726849
Starting period 1 | Past days: 389
Listed stocks for the period: 60


100%|██████████████████████████████████████████████████████████████████████████████████| 59/59 [00:00<00:00, 135.04it/s]


SSE shape: (60, 60)
Minimum SSE = 2.034667646393423
Minimum SSE = 2.1760679753098473
Minimum SSE = 2.295011452873681
Minimum SSE = 2.4118901056766444
Minimum SSE = 2.6992165314117873
Starting period 2 | Past days: 521
Listed stocks for the period: 60


...

SSE shape: (121, 121)
Minimum SSE = 0.7248298102637717
Minimum SSE = 0.9104854639620256
Minimum SSE = 0.9481931717109777
Minimum SSE = 1.0342465130971803
Minimum SSE = 1.2896771641885536
Starting period 53 | Past days: 7173
Listed stocks for the period: 121


100%|█████████████████████████████████████████████████████████████████████████████████| 120/120 [00:01<00:00, 65.86it/s]


SSE shape: (121, 121)
Minimum SSE = 0.5243612833387602
Minimum SSE = 0.5940403594466453
Minimum SSE = 0.9281844820108459
Minimum SSE = 0.9567485942302445
Minimum SSE = 1.03996106873001
Starting period 54 | Past days: 7304
Listed stocks for the period: 118


100%|█████████████████████████████████████████████████████████████████████████████████| 117/117 [00:01<00:00, 63.38it/s]

SSE shape: (118, 118)
Minimum SSE = 0.5493329029760822
Minimum SSE = 0.5948726473896642
Minimum SSE = 0.7130523638934851
Minimum SSE = 0.8470306460063916
Minimum SSE = 1.048284408360507


## Testando Lucro da estratégia

In [8]:
pairs

,Stock A,Stock B,SSE,Semestre
0,PETR4,BBDC4,1.147011,0
1,ITUB4,CMIG3,1.229207,0
2,ITSA4,POMO4,1.417282,0
3,BBDC3,CRUZ3,1.469926,0
4,CMIG4,CRUZ3,1.676307,0
...,...,...,...,...
270,ENBR3,TRPL4,0.549333,54
271,USIM5,USIM3,0.594873,54
272,ELET3,ELET6,0.713052,54
273,VIVT3,EGIE3,0.847031,54


In [9]:
# Vendo o Lucro da estratégia
past_days = 0
resumos = []

for big_loop in range(0, len(semestres) - 3):
    print(f"Starting period {big_loop} | Past days: {past_days}")

    pairs_sem = pairs[pairs['Semestre'] == big_loop]

    # Listando os dias dos intervalos
    inicio = (Datas == semestres['Data'][big_loop]).query("Data == True").index[0]
    twelve_months = (Datas == semestres['Data'][big_loop + 2]).query("Data == True").index[0]
    six_months = (Datas == semestres['Data'][big_loop + 3]).query("Data == True").index[0]

    for i in range(0,pairs_sem.shape[0]):
        S1_name = pairs_sem.iloc[i, 0]
        S2_name = pairs_sem.iloc[i, 1]
        SSE = pairs_sem.iloc[i, 2]

        print(f"S1: {S1_name} | S2: {S2_name} | SSE: {SSE}")

        #Aqui tá o problema das datas:
        S1 = log_data[S1_name].iloc[twelve_months:six_months] # periodo de teste
        S2 = log_data[S2_name].iloc[twelve_months:six_months] # periodo de teste

        #spread, convertendo Datas para nºs de dias
        spread_ = S1 - S2
        spread_.name = "spread"
        spread = pd.merge(spread_,Datas, on="Data")
        spread.index = spread['indice']
        spread.drop(['Data','indice'],inplace=True,axis=1)

        std_limit = 2*np.std(spread)  # standard deviation

        # Pegando o resultado da estratégia
        ret, resumos = calculate_profit(i, spread, std_limit[0], S1_name, S2_name, resumos, big_loop)
    print("-------------------------------------------------")

Starting period 0 | Past days: 0
S1: PETR4 | S2: BBDC4 | SSE: 1.1470111572247645
Par PETR4-BBDC4 sem convergência, retorno_op: -0.08188873857769119
S1: ITUB4 | S2: CMIG3 | SSE: 1.2292074671340252
Par ITUB4-CMIG3 sem convergência, retorno_op: 0.1923893994394017
S1: ITSA4 | S2: POMO4 | SSE: 1.4172821904116257
Par ITSA4-POMO4 sem convergência, retorno_op: -0.05813494423065757
S1: BBDC3 | S2: CRUZ3 | SSE: 1.469926115829889
S1: CMIG4 | S2: CRUZ3 | SSE: 1.6763074444726849
-------------------------------------------------
Starting period 1 | Past days: 0
S1: ITSA4 | S2: POMO4 | SSE: 2.034667646393423
Par ITSA4-POMO4 sem convergência, retorno_op: -0.06257964793901127
S1: BBDC3 | S2: AMBV4 | SSE: 2.1760679753098473
S1: ELET6 | S2: WHMT3 | SSE: 2.295011452873681
Par ELET6-WHMT3 sem convergência, retorno_op: -0.31327460769384596
S1: CMIG4 | S2: AMBV4 | SSE: 2.4118901056766444
Par CMIG4-AMBV4 sem convergência, retorno_op: -0.08778375919327075
S1: BBDC3 | S2: CMIG4 | SSE: 2.6992165314117873
Par BBD

In [11]:
#Salvando os resultados:

cols = ['Operação', 'Semestre', 'Abertura', 'Fechamento', 'Dias', 'Retorno total', 'Ticker 1', 'Ticker 2', 'Converged']
df = pd.DataFrame(resumos, columns = cols)
df['Index'] = df['Ticker 1'].astype(str) + '-' + df['Ticker 2'].astype(str) + '-' + df['Operação'].astype(str)
df['Retorno total - exp'] = np.exp(df['Retorno total'])

#Drive
'''
pasta2 = str("/content/drive/MyDrive/Python/TCC-Cloud/Distancia-Cointegração/result_distancia/")
print(pasta2)
'''

#PC:
pasta2 = str(pasta + "/Distancia-Cointegração/result_distancia/")
print(pasta2)

#os.makedirs(pasta2, exist_ok=True)
df.to_csv(pasta2 + "operations_D5.csv", sep=',', index=False)

/mnt/c/Users/MarceloPolecram/Python/MEU_TCC/Distancia-Cointegração/result_distancia/


## Rascunhos